<font size="-0.1">

**<center><h1>Exploring Reinforcement Learning Methods to approximate the Lunar Lander problem</h1></center>**
**<center><h2>Advanced Reinforcement Learning Assignment (Actor Critic Experiment)</h2></center>**
**<center><h3>Matthias Bartolo</h3></center>**

</font>

**<h5>Package installation</h5>**

<font size="-0.1">

```pip
conda install swig
conda install nomkl
pip install gymnasium[all]
pip install ufal.pybox2d
pip install pygame
pip install renderlab
pip install numpy
pip install matplotlib
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118/torch_stable.html
```

</font>

**<h5>Package imports</h5>**

In [37]:
import os
import time
import renderlab as rl
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import AdamW
%matplotlib inline
import matplotlib.pyplot as plt
from typing import Sequence
from collections import namedtuple, deque
import itertools
import random
import warnings
warnings.filterwarnings("ignore")

# Setting the device to cpu as it was faster than gpu for this task
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("CUDA is available, device is set to {}".format(device))

CUDA is available, device is set to cuda:0


**<h5>Declaring Hyperparameters</h5>**

In [39]:
# The learning rate α ∈ (0, 1] controls how much we update our current value estimates towards newly received returns.
ALPHA = 0.0003
# Gamma refers to the discount factor γ ∈ [0, 1]. It quantifies how much importance is given to future rewards.
GAMMA = 0.99
# The batch size is the number of training examples used in one iteration (that is, one gradient update) of training.
BATCH_SIZE = 128
# The buffer size is the number of transitions stored in the replay buffer, which the agent samples from to learn.
BUFFER_SIZE = 10000
# The minimum replay size is the minimum number of transitions that need to be stored in the replay buffer before the agent starts learning.
MIN_REPLAY_SIZE = 5000
# The maximum replay size is the maximum number of transitions that can be stored in the replay buffer.
MAX_REPLAY_SIZE = 50
# Episode start, episode end and episode decay are the parameters for the epsilon greedy policy.
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 0.997
# The target update frequency is the frequency with which the target network is updated.
TARGET_UPDATE_FREQ = 5
# The success criteria is the average reward over the last 50 episodes that the agent must achieve to be considered successful.
SUCCESS_CRITERIA = 195
# The number of environments to run in parallel
NUM_ENVS = 64
# The number of neurons in the hidden layer
HIDDEN_SIZE = 64

**<h5>Environment Setup</h5>**

In [31]:
# Creating the "LunarLanderContinuous-v2" environment from gymnasium with the render mode set to rgb_array and the number of environments set to NUM_ENVS
env = gym.vector.make("LunarLanderContinuous-v2", render_mode="rgb_array", num_envs=NUM_ENVS, asynchronous=False)

# Resetting the environment
env.reset()

(array([[ 2.53057480e-03,  1.42091250e+00,  2.56304681e-01,
          4.44108158e-01, -2.92551960e-03, -5.80568425e-02,
          0.00000000e+00,  0.00000000e+00],
        [ 2.19755177e-03,  1.39875138e+00,  2.22572923e-01,
         -5.40838003e-01, -2.53961608e-03, -5.04160933e-02,
          0.00000000e+00,  0.00000000e+00],
        [-1.60093303e-03,  1.40937567e+00, -1.62177622e-01,
         -6.86453208e-02,  1.86193432e-03,  3.67356613e-02,
          0.00000000e+00,  0.00000000e+00],
        [-3.01208487e-03,  1.41277552e+00, -3.05111289e-01,
          8.24583247e-02,  3.49707413e-03,  6.91122040e-02,
          0.00000000e+00,  0.00000000e+00],
        [ 6.09970093e-03,  1.39992833e+00,  6.17814660e-01,
         -4.88537729e-01, -7.06121139e-03, -1.39944300e-01,
          0.00000000e+00,  0.00000000e+00],
        [-3.57818615e-04,  1.41103745e+00, -3.62634584e-02,
          5.21482015e-03,  4.21458215e-04,  8.21423065e-03,
          0.00000000e+00,  0.00000000e+00],
        [-4.0388

**<h5>Visualising Observation Space</h5>**

In [34]:
# Checking the observation space
obs, info = env.reset()

print("The observation space consists of a Box of shape {}".format(obs.shape))

# Setting observation space to the first observation
obs = obs[0]
print('\033[1m' + "A single observation space consists of the following values:" + '\033[0m')
print('\033[37m' + 'X position: ' + '\033[0m' + obs[0].astype(str))
print('\033[36m' + 'Y position: ' + '\033[0m' + obs[1].astype(str))
print('\033[35m' + 'X velocity: ' + '\033[0m' + obs[2].astype(str))
print('\033[34m' + 'Y velocity: ' + '\033[0m' + obs[3].astype(str))
print('\033[33m' + 'Angle: ' + '\033[0m' + obs[4].astype(str))
print('\033[32m' + 'Angular velocity: ' + '\033[0m' + obs[5].astype(str))
print('\033[31m' + 'Left leg touching the ground (0 or 1): ' + '\033[0m' + obs[6].astype(str))
print('\033[1m' + 'Right leg touching the ground (0 or 1): ' + '\033[0m' + obs[7].astype(str))

The observation space consists of a Box of shape (64, 8)
A single observation space consists of the following values:
X position: 0.0026765824
Y position: 1.4040436
X velocity: 0.27109635
Y velocity: -0.305629
Angle: -0.003094715
Angular velocity: -0.061407186
Left leg touching the ground (0 or 1): 0.0
Right leg touching the ground (0 or 1): 0.0


**<h5>Visualising Action Space</h5>**

In [36]:
# Checking the action space
action_space = env.action_space

print("The action space consists of a Box of shape {}".format(action_space.shape))
print('\033[1m' + "The action space consists of the following values:" + '\033[0m')
print('\033[35m' + 'Value 1 (Thrust): (-1.0, 1.0) for the main engine, whereby: ' + '\033[0m')
print('\033[35m' + '[-1.0, -0.5] is off' + '\033[0m')
print('\033[35m' + '[0.5, 1.0] is on' + '\033[0m')
print()
print('\033[34m' + 'Value 2 (Rotation): (-1.0, 1.0) whereby:' + '\033[0m')
print('\033[34m' + '[-1.0, -0.5] is left' + '\033[0m')
print('\033[34m' + '[0.5, 1.0] is right' + '\033[0m')
print('\033[34m' + '[-0.5, 0.5] is off' + '\033[0m')

The action space consists of a Box of shape (64, 2)
The action space consists of the following values:
Value 1 (Thrust): (-1.0, 1.0) for the main engine, whereby: 
[-1.0, -0.5] is off
[0.5, 1.0] is on

Value 2 (Rotation): (-1.0, 1.0) whereby:
[-1.0, -0.5] is left
[0.5, 1.0] is right
[-0.5, 0.5] is off


**<h5>Actor Network</h5>**

In [40]:
class ActorNet(nn.Module):
    """
        Actor network for the PPO algorithm

        Args:
            input_size (int): The number of input features
            hidden_units (int): The number of hidden units
            output_size (int): The number of output features

        Layers:
            1. Linear layer with input_size and hidden_units
            2. Tanh activation function
            3. Linear layer with hidden_units and int(hidden_units/2)
            4. Tanh activation function
            5. Linear layer with int(hidden_units/2) and output_size
            6. Linear layer with int(hidden_units/2) and output_size
        
    """
    def __init__(self, input_size, hidden_units=64, output_size=2):
        super(ActorNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_units),
            nn.Tanh(),
            nn.Linear(hidden_units, int(hidden_units/2)),
            nn.Tanh()
        )

        # The mu_head is the mean of the normal distribution
        self.mu_head = nn.Linear(int(hidden_units/2),  output_size)

        # The logstd_head is the standard deviation of the normal distribution
        self.logstd_head = nn.Linear(int(hidden_units/2),  output_size)

    def forward(self, x):
        x = self.model(x)

        # The mean is scaled by 2 and the standard deviation is exponentiated
        loc = torch.tanh(self.mu_head(x)) * 2

        # The standard deviation is exponentiated
        scale = torch.exp(self.logstd_head(x))

        # Returning the mean and standard deviation
        return loc, scale

    def __call__(self, x):
        out = self.forward(x)
        return out

**<h5>Critic Network</h5>**

In [41]:
class CriticNet(nn.Module):
    """
        Critic network for the PPO algorithm

        Args:
            input_size (int): The number of input features
            hidden_units (int): The number of hidden units

        Layers:
            1. Linear layer with input_size and hidden_units
            2. Tanh activation function
            3. Linear layer with hidden_units and int(hidden_units/2)
            4. Tanh activation function
            5. Linear layer with int(hidden_units/2) and output_size
        
    """
    def __init__(self, input_size, hidden_units=64):
        super(CriticNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_units),
            nn.Tanh(),
            nn.Linear(hidden_units, int(hidden_units/2)),
            nn.Tanh()
        )

        # The value_head is the value of the state action pair
        self.value_head = nn.Linear(int(hidden_units/2), 1)

    def forward(self, x):
        x = self.model(x)
        value = self.value_head(x)
        return value

    def __call__(self, x):
        out = self.forward(x)
        return out

**<h5>PPO Agent</h5>**

In [ ]:
class PPOAgent():
    def __init__(self, env, running_memory, actor_net, critic_net, hidden_size=HIDDEN_SIZE, actor_lr=ALPHA, critic_lr=ALPHA, actor_optimizer=AdamW, critic_optimizer=AdamW, criterion=nn.SmoothL1Loss(), name="PPO"):
        self.env = env
        self.running_memory = running_memory
        self.ninputs = env.observation_space.shape[0]
        self.noutputs = env.action_space.shape[0]
        self.hidden_size = hidden_size
        self.actor_lr = actor_lr
        self.critic_lr = critic_lr
        self.criterion = criterion
        self.actor_net = actor_net(self.ninputs, self.hidden_size, self.noutputs).to(device)
        self.critic_net = critic_net(self.ninputs, self.hidden_size).to(device)

        # Applying the initialize_weights function to the actor and critic networks
        self.actor_net.apply(self.initialize_weights)
        self.critic_net.apply(self.initialize_weights)

        # Creating the optimizer for the actor and critic networks
        self.actor_optimizer = actor_optimizer(self.actor_net.parameters(), lr=self.actor_lr)
        self.critic_optimizer = critic_optimizer(self.critic_net.parameters(), lr=self.critic_lr)

        self.epsilon = EPS_START
        self.steps_done = 0
        self.episodes = 0
        self.episode_info = {"name":name, "episode_avg_rewards": [], "episode_lengths": [], "best_episode": {"episode": 0, "avg_reward": np.NINF}, "solved": False, "eps_duration": 0}
        self.display_every_n_episodes = 100
        
    def initialize_weights(self, layer):
        """
            Initializes the weights of the layers of the actor and critic networks

            Args:
                layer (nn.Module): The layer to initialize the weights of
        """
        # Checking if the layer is a linear layer
        if isinstance(layer, nn.Conv2d):
            # Initializing the weights of the layer using the Xavier normal initialization
            nn.init.xavier_normal_(layer.weight.data, nonlinearity='relu')

            # Initializing the bias of the layer to 0 if it exists
            if layer.bias is not None:
                # Initializing the bias of the layer to 0
                nn.init.constant_(layer.bias.data, 0)

        # Checking if the layer is a batch normalization layer
        elif isinstance(layer, nn.BatchNorm2d):
            # Initializing the weights of the layer to 1
            nn.init.constant_(layer.weight.data, 1)

            # Initializing the bias of the layer to 0
            nn.init.constant_(layer.bias.data, 0)

        # Checking if the layer is a linear layer
        elif isinstance(layer, nn.Linear):
            # Initializing the weights of the layer using the Xavier normal initialization
            nn.init.xavier_normal_(layer.weight.data)

            # Initializing the bias of the layer to 0 if it exists
            nn.init.constant_(layer.bias.data, 0)

    def save_model(self, path):
        """
            Saves the model

            Args:
                path (str): The path to save the model to
        """
        # Creating the directory if it does not exist
        if not os.path.exists(path):
            os.makedirs(path)

        # Saving the actor and critic networks
        torch.save(self.actor_net.state_dict(), path + "/actor_net.pt")
        torch.save(self.critic_net.state_dict(), path + "/critic_net.pt")

    def load_model(self, path):
        """
            Loads the model

            Args:
                path (str): The path to load the model from
        """
        # Loading the actor and critic networks
        self.actor_net.load_state_dict(torch.load(path + "/actor_net.pt"))
        self.critic_net.load_state_dict(torch.load(path + "/critic_net.pt"))

    def get_episode_info(self):
        """Returns the episode info """
        return self.episode_info

In [ ]:
class DQNAgent():
    """
        The DQN agent that interacts with the environment

        Args:
            env: The environment to interact with
            replay_buffer: The replay buffer to store and sample transitions from
            target_update_freq: The frequency with which the target network is updated
            criterion: The loss function used to train the policy network
            name: The name of the agent (default: DQN)
            network: The network used to estimate the action-value function (default: DQN)

        Attributes:
            env: The environment to interact with
            replay_buffer: The replay buffer to store and sample transitions from
            nsteps: The number of steps to run the agent for
            target_update_freq: The frequency with which the target network is updated
            ninputs: The number of inputs
            noutputs: The number of outputs
            policy_net: The policy network
            target_net: The target network
            optimizer: The optimizer used to update the policy network
            criterion: The loss function used to train the policy network
            epsilon: The probability of selecting a random action
            steps_done: The number of steps the agent has run for
            episodes: The number of episodes the agent has run for
            episode_avg_rewards: The average reward for each episode
            episode_lengths: The lengths of each episode
            best_episode: The best episode
            solved: Whether the environment is solved
            display_every_n_episodes: The number of episodes after which the results are displayed
            time: The time taken to run the agent
    """
    def __init__(self, env, replay_buffer, target_update_freq, criterion=nn.SmoothL1Loss(), name="DQN", network=DQN):
        self.env = env
        self.replay_buffer = replay_buffer
        self.target_update_freq = target_update_freq
        self.ninputs = env.observation_space.shape[0]
        self.noutputs = env.action_space.n
        self.policy_net = network(self.ninputs, self.noutputs).to(device)
        self.target_net = network(self.ninputs, self.noutputs).to(device)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        self.target_net.eval()
        self.optimizer = torch.optim.Adam(self.policy_net.parameters(), lr=ALPHA)
        self.criterion = criterion
        self.epsilon = EPS_START
        self.steps_done = 0
        self.episodes = 0
        self.episode_info = {"name":name, "episode_avg_rewards": [], "episode_lengths": [], "best_episode": {"episode": 0, "avg_reward": np.NINF}, "solved": False, "eps_duration": 0}
        self.display_every_n_episodes = 100

    def select_action(self, state):
        """ Selects an action using an epsilon greedy policy """
        # Selecting a random action with probability epsilon
        if random.random() <= self.epsilon: # Exploration
            action = self.env.action_space.sample()
        else: # Exploitation
            # Selecting the action with the highest Q-value otherwise
            with torch.no_grad():
                state = torch.from_numpy(state).float().unsqueeze(0).to(device)
                qvalues = self.policy_net(state)
                action = qvalues.argmax().item()
        return action
        
    def update(self):
        """ Updates the policy network using a batch of transitions """
        # Sampling a batch of transitions from the replay buffer
        states, actions, rewards, dones, next_states = self.replay_buffer.sample_batch()

        # Converting the tensors to cuda tensors
        states = states.to(device)
        actions = actions.to(device)
        rewards = rewards.to(device)
        dones = dones.to(device)
        next_states = next_states.to(device)
        
        # Calculating the Q-values for the current states
        qvalues = self.policy_net(states).gather(1, actions)
        
        # Calculating the Q-values for the next states
        with torch.no_grad():
            # Calculating the Q-values for the next states using the target network (Q(s',a'))
            target_qvalues = self.target_net(next_states)

            # Calculating the maximum Q-values for the next states (max(Q(s',a'))
            max_target_qvalues = torch.max(target_qvalues, axis=1).values.unsqueeze(1)

            # Calculating the next Q-values using the Bellman equation (Q(s,a) = r + γ * max(Q(s',a')))
            next_qvalues = rewards + GAMMA * (1 - dones.type(torch.float32)) * max_target_qvalues

        # Calculating the loss
        loss = self.criterion(qvalues, next_qvalues)

        # Optimizing the model
        self.optimizer.zero_grad()
        loss.backward()

        # Clipping the gradients
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()

        # Updating the target network
        if self.episodes % self.target_update_freq == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
            

    def update_epsilon(self):
        """ Updates epsilon """
        self.epsilon = max(EPS_END, EPS_DECAY * self.epsilon)

    def train(self):
        """ Trains the agent for nsteps steps """
        # Resetting the environment
        obs, _ = self.env.reset()

        # Retrieving the starting time
        start_time = time.time()

        # Setting the episode_reward to 0
        episode_reward = 0

        # Running the agent for nsteps steps
        for step in itertools.count():
            # Selecting an action
            action = self.select_action(obs)

            # Taking a step in the environment
            new_obs, reward, terminated, truncated, _ = self.env.step(action)

            # Adding the reward to the cumulative reward
            episode_reward += reward

            # Setting done to terminated or truncated
            done = terminated or truncated

            # Creating a transition
            transition = Transition(obs, action, reward, done, new_obs)

            # Appending the transition to the replay buffer
            self.replay_buffer.append(transition)

            # Resetting the observation
            obs = new_obs

            # Ending the episode and displaying the results if the episode is done
            if done:
                # Appending the rewards to the replay buffer
                self.replay_buffer.rewards.append(episode_reward)

                # Updating epsilon
                self.update_epsilon()

                # Resetting the environment
                obs, _ = self.env.reset()

                # Incrementing the number of episodes
                self.episodes += 1

                # Appending the average episode reward
                self.episode_info["episode_avg_rewards"].append(np.mean(self.replay_buffer.rewards))

                # Appending the episode length
                self.episode_info["episode_lengths"].append(self.steps_done)

                # Updating the best episode
                if self.episode_info["episode_avg_rewards"][-1] > self.episode_info["best_episode"]["avg_reward"]:
                    self.episode_info["best_episode"]["episode"] = self.episodes
                    self.episode_info["best_episode"]["avg_reward"] = self.episode_info["episode_avg_rewards"][-1]

                # Checking if the environment is solved
                if np.mean(self.episode_info["episode_avg_rewards"][-MAX_REPLAY_SIZE:]) >= SUCCESS_CRITERIA:
                    self.episode_info["solved"] = True

                # Checking if the environment is solved
                if self.episode_info["solved"]:
                    print("\033[32mSolved in {} episodes!\033[0m".format(self.episodes))
                    print("-" * 100)
                    break
                
                # Displaying the results
                if self.episodes % self.display_every_n_episodes == 0:
                    print("\033[35mEpisode:\033[0m {} \033[35mEpsilon:\033[0m {:.2f} \033[35mAverage Reward:\033[0m {} \033[35mEpisode Length:\033[0m {}".format(
                                self.episodes,
                                self.epsilon,
                                self.episode_info["episode_avg_rewards"][-1],
                                self.episode_info["episode_lengths"][-1])
                        )
                    print("-" * 100)

                # Resetting the cumulative reward
                episode_reward = 0

            # Updating the policy network
            self.update()

            # Updating the number of steps
            self.steps_done += 1

        # Retrieving the ending time
        end_time = time.time()

        # Calculating the time taken
        self.episode_info["eps_duration"] = end_time - start_time

    def run(self):
        """ Runs the agent """
        # Initializing the replay buffer
        self.replay_buffer.initialize()

        # Training the agent
        self.train()


    def test(self):
        """ Tests the trained agent """
        # Resetting the environment
        obs, _ = self.env.reset()

        # Playing the environment
        while True:
            # Selecting the action with the highest Q-value
            action = int(torch.argmax(self.policy_net(torch.from_numpy(obs).float().unsqueeze(0).to(device))).item())

            # Taking a step in the environment
            obs, _, terminated, truncated, _ = self.env.step(action)

            # Setting done to terminated or truncated
            if terminated or truncated:
                break
        
        # Playing the video
        env.play()


    def save(self, path="models/dqn"):
        """ Function to save the model 
            
            Args:
                path (str): The path to save the model to
        """
        # Creating the directory if it does not exist
        if not os.path.exists(path):
            os.makedirs(path)

        # Saving the model
        torch.save(self.policy_net.state_dict(), path + "/policy_net.pth")
        torch.save(self.target_net.state_dict(), path + "/target_net.pth")

        # Saving the episode info
        np.save(path + "/episode_info.npy", self.episode_info)

    def load(self, path="models/dqn"):
        """ Function to load the model 
            
            Args:
                path (str): The path to load the model from
        """
        # Loading the model
        self.policy_net.load_state_dict(torch.load(path + "/policy_net.pth"))
        self.target_net.load_state_dict(torch.load(path + "/target_net.pth"))

        # Loading the episode info
        self.episode_info = np.load(path + "/episode_info.npy", allow_pickle=True).item()

    def get_episode_info(self):
        """ Returns the episode info """
        return self.episode_info

In [23]:
# env.close()